In [0]:
df_results = spark.read.table('hive_metastore.default.results')
df_goalscorers = spark.read.table('hive_metastore.default.goalscorers')

In [0]:
df_results.head()

Row(date=datetime.date(1872, 11, 30), home_team='Scotland', away_team='England', home_score=0, away_score=0, tournament='Friendly', city='Glasgow', country='Scotland', neutral=False)

In [0]:
df_goalscorers.head()

Row(date=datetime.date(1916, 7, 2), home_team='Chile', away_team='Uruguay', team='Uruguay', scorer='José Piendibene', minute='44', own_goal=False, penalty=False)

### 1. Número de goles que ha marcado Lionel Messi (sin contar goles en propia puerta).

In [0]:
df_goalscorers.createOrReplaceTempView('goalscorers')
spark.sql('SELECT COUNT(*) AS goals_from_messi FROM goalscorers WHERE scorer = "Lionel Messi" AND own_goal = "FALSE"').show()


+----------------+
|goals_from_messi|
+----------------+
|              55|
+----------------+



### 2. Listado de los 5 partidos más recientes que ha jugado la selección española.

In [0]:
df_results.createOrReplaceTempView("goalscorers")
spark.sql("SELECT * FROM goalscorers WHERE home_team = 'Spain' OR away_team = 'Spain' ORDER BY date DESC LIMIT 5").show()
  

+----------+-----------+-----------+----------+----------+-------------------+--------------------+-----------+-------+
|      date|  home_team|  away_team|home_score|away_score|         tournament|                city|    country|neutral|
+----------+-----------+-----------+----------+----------+-------------------+--------------------+-----------+-------+
|2025-03-23|      Spain|Netherlands|         3|         3|UEFA Nations League|            Valencia|      Spain|  false|
|2025-03-20|Netherlands|      Spain|         2|         2|UEFA Nations League|           Rotterdam|Netherlands|  false|
|2024-11-18|      Spain|Switzerland|         3|         2|UEFA Nations League|Santa Cruz de Ten...|      Spain|  false|
|2024-11-15|    Denmark|      Spain|         1|         2|UEFA Nations League|          Copenhagen|    Denmark|  false|
|2024-10-15|      Spain|     Serbia|         3|         0|UEFA Nations League|             Cordoba|      Spain|  false|
+----------+-----------+-----------+----

### 3. Número de goles que ha marcado España en toda su historia. Esta información debe extraerse del archivo results.csv, ya que goalscorers.csv no contiene todos los goles.

In [0]:
df_results.createOrReplaceTempView('results')
spark.sql('''
      SELECT 
        SUM(CASE WHEN home_team = 'Spain' THEN home_score ELSE 0 END) +
        SUM(CASE WHEN away_team = 'Spain' THEN away_score ELSE 0 END) AS total_goals_from_spain
      FROM results
''').show()

+----------------------+
|total_goals_from_spain|
+----------------------+
|                  1567|
+----------------------+



### 4. Listado de los 5 máximos goleadores con la selección española (sin contar goles en propia puerta).

In [0]:
df_goalscorers.createOrReplaceTempView('goalscorers')
spark.sql('''
    SELECT scorer, COUNT(*) AS goals
    FROM goalscorers
    WHERE team = 'Spain' AND own_goal = 'FALSE'
    GROUP BY scorer
    ORDER BY goals DESC
    LIMIT 5
''').show()

+---------------+-----+
|         scorer|goals|
+---------------+-----+
|    David Villa|   41|
|           Raúl|   32|
|  Álvaro Morata|   29|
|Fernando Torres|   28|
|Fernando Hierro|   25|
+---------------+-----+



### 5. Listado de los jugadores españoles que han marcado algún gol de penalti en una Eurocopa (UEFA Euro), ordenados alfabéticamente.

In [0]:
df_goalscorers.createOrReplaceTempView('goalscorers')
df_results.createOrReplaceTempView('reults')
spark.sql('''
    SELECT DISTINCT scorer
    FROM results r
    JOIN goalscorers g
    ON r.date = g.date AND r.home_team = g.home_team AND r.away_team = g.away_team
    WHERE g.team = 'Spain' AND g.penalty = 'TRUE' AND r.tournament LIKE '%Euro%'
    ORDER BY scorer
''').show()

+--------------------+
|              scorer|
+--------------------+
|      Andrés Iniesta|
|         Daniel Ruiz|
|         David Villa|
|     Fernando Hierro|
|Francisco José Ca...|
|     Gaizka Mendieta|
|      José Claramunt|
|  Juan Antonio Señor|
|              Míchel|
|               Pirri|
|        Sergio Ramos|
|         Xabi Alonso|
|       Álvaro Morata|
+--------------------+



### 6. Listado de los 5 máximos goleadores de las fases finales de los mundiales (FIFA World Cup) (sin contar goles en propia puerta).

In [0]:
df_goalscorers.createOrReplaceTempView('goalscorers')
df_results.createOrReplaceTempView('reults')
spark.sql('''
    SELECT scorer, COUNT(*) AS goals
    FROM results r
    JOIN goalscorers g
    ON r.date = g.date AND r.home_team = g.home_team AND r.away_team = g.away_team
    WHERE r.tournament = 'FIFA World Cup' AND g.own_goal = 'FALSE'
    GROUP BY scorer
    ORDER BY goals DESC
    LIMIT 5
''').show()

+--------------+-----+
|        scorer|goals|
+--------------+-----+
|Miroslav Klose|   16|
|       Ronaldo|   15|
|   Gerd Müller|   14|
|  Lionel Messi|   13|
| Just Fontaine|   13|
+--------------+-----+

